# Database operations low level

## Creating the databases

Data source: https://www.imdb.com/

### IMDb IDs

IMDb uses unique identifiers for each of the entities referenced in IMDb data. For example "Name IDs" identify name entities (people) and "Title IDs" identify title entities (movies, series, episodes and video games). IMDb identifiers always take the form of two letters, which signify the type of entity being identified, followed by a sequence of at least seven numbers that uniquely identify a specific entity of that type. For example:


* tt0050083 is the unique identifier for the movie "12 Angry Men (1957)", where tt signifies that it's a title entity and 0050083 uniquely indicates "12 Angry Men (1957)".
    
* nm0000020 is the unique identifier for the actor "Henry Fonda", where nm signifies that it's a name entity and 0000020 uniquely indicates "Henry Fonda".


In [1]:
from prettytable import PrettyTable
from typing import Callable

movies = PrettyTable()
movies.field_names = ['IMDb-id', 'title', 'year', 'IMDb-score']
movies.add_row(['tt0468569', 'The Dark Knight', '2008', '9.0'])
movies.add_row(['tt0137523', 'Fight Club', '1999', '8.8'])
movies.add_row(['tt0133093', 'Matrix', '1999', '8.7'])
movies.add_row(['tt1375666', 'Inception', '2010', '8.8'])
movies.add_row(['tt0816692', 'Interstellar', '2014', '8.7'])
movies.add_row(['tt4154756', 'Avengers: Infinity War', '2018', '8.4'])
movies.add_row(['tt0120338', 'Titanic', '1997', '7.9'])
movies.add_row(['tt0499549', 'Avatar', '2009', '7.9'])
print(movies)

+-----------+------------------------+------+------------+
|  IMDb-id  |         title          | year | IMDb-score |
+-----------+------------------------+------+------------+
| tt0468569 |    The Dark Knight     | 2008 |    9.0     |
| tt0137523 |       Fight Club       | 1999 |    8.8     |
| tt0133093 |         Matrix         | 1999 |    8.7     |
| tt1375666 |       Inception        | 2010 |    8.8     |
| tt0816692 |      Interstellar      | 2014 |    8.7     |
| tt4154756 | Avengers: Infinity War | 2018 |    8.4     |
| tt0120338 |        Titanic         | 1997 |    7.9     |
| tt0499549 |         Avatar         | 2009 |    7.9     |
+-----------+------------------------+------+------------+


In [2]:
actors = PrettyTable()
actors = PrettyTable()
actors.field_names = ['id', 'name']
actors.add_row(['nm0000288', 'Christian Bale'])
actors.add_row(['nm0000093', 'Brad Pitt'])
actors.add_row(['nm0000138', 'Leonardo DiCaprio'])
actors.add_row(['nm0000701', 'Kate Winslet'])
print(actors)

+-----------+-------------------+
|     id    |        name       |
+-----------+-------------------+
| nm0000288 |   Christian Bale  |
| nm0000093 |     Brad Pitt     |
| nm0000138 | Leonardo DiCaprio |
| nm0000701 |    Kate Winslet   |
+-----------+-------------------+


In [3]:
cast = PrettyTable()
cast.field_names = ['movie_id', 'person_id']
cast.add_row(['tt0468569', 'nm0000288']) # Christian Bale in The Dark Knight
cast.add_row(['tt0137523', 'nm0000093']) # Brad Pitt in Fight Club
cast.add_row(['tt1375666', 'nm0000138']) # Leonardo DiCaprio in Inception
cast.add_row(['tt0120338', 'nm0000138']) # Leonardo DiCaprio in Titanic
cast.add_row(['tt0120338', 'nm0000701']) # Kate Winslet in Titanic
print(cast)

+-----------+-----------+
|  movie_id | person_id |
+-----------+-----------+
| tt0468569 | nm0000288 |
| tt0137523 | nm0000093 |
| tt1375666 | nm0000138 |
| tt0120338 | nm0000138 |
| tt0120338 | nm0000701 |
+-----------+-----------+


In [4]:
director = PrettyTable()
director.field_names = ['id', 'name', 'movie_id']
director.add_row(['nm0634240', 'Christopher Nolan', 'tt0816692']) # Christopher Nolan directed Interstellar
director.add_row(['nm0634240', 'Christopher Nolan', 'tt0468569']) # Chritopher Nolan directed The Dark Knight
director.add_row(['nm0000116', 'James Cameron', 'tt0120338']) # James Cameron directed Titanic
director.add_row(['nm0000116', 'James Cameron', 'tt0499549']) # James Cameron directed Avatar
print(director)

+-----------+-------------------+-----------+
|     id    |        name       |  movie_id |
+-----------+-------------------+-----------+
| nm0634240 | Christopher Nolan | tt0816692 |
| nm0634240 | Christopher Nolan | tt0468569 |
| nm0000116 |   James Cameron   | tt0120338 |
| nm0000116 |   James Cameron   | tt0499549 |
+-----------+-------------------+-----------+


## Operation: Project

In [5]:
def project(table: PrettyTable, cols: list[int], distinct: bool=False) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = [table.field_names[col_index] for col_index in cols]
    if not distinct:
        for row in table.rows:
            table_result.add_row([row[col_index] for col_index in cols])
        return table_result
    for row in table.rows:
        row_result = [row[col_index] for col_index in cols]
        if row_result not in table_result.rows:
            table_result.add_row(row_result)
    return table_result


``` sql
SELECT title, IMDb-score FROM movies;
```

In [6]:
print(project(movies, [1, 3]))

+------------------------+------------+
|         title          | IMDb-score |
+------------------------+------------+
|    The Dark Knight     |    9.0     |
|       Fight Club       |    8.8     |
|         Matrix         |    8.7     |
|       Inception        |    8.8     |
|      Interstellar      |    8.7     |
| Avengers: Infinity War |    8.4     |
|        Titanic         |    7.9     |
|         Avatar         |    7.9     |
+------------------------+------------+


``` sql
SELECT title FROM movies;
```

In [7]:
print(project(movies, [1]))

+------------------------+
|         title          |
+------------------------+
|    The Dark Knight     |
|       Fight Club       |
|         Matrix         |
|       Inception        |
|      Interstellar      |
| Avengers: Infinity War |
|        Titanic         |
|         Avatar         |
+------------------------+


## Operation: Select

In [8]:
def select(table: PrettyTable, phi: Callable[[list], bool]) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table.field_names
    for row in table.rows: 
        if phi(row):
            table_result.add_row(row)
    return table_result 

```sql
SELECT * FROM movies
WHERE IMDb-score >= 8.8;
```

In [9]:
print(select(movies, lambda row: row[3] >= '8.8'))

+-----------+-----------------+------+------------+
|  IMDb-id  |      title      | year | IMDb-score |
+-----------+-----------------+------+------------+
| tt0468569 | The Dark Knight | 2008 |    9.0     |
| tt0137523 |    Fight Club   | 1999 |    8.8     |
| tt1375666 |    Inception    | 2010 |    8.8     |
+-----------+-----------------+------+------------+


``` sql
SELECT * FROM movies
WHERE year > 2010;
```

In [10]:
print(select(movies, lambda row: row[2] >= '2010'))

+-----------+------------------------+------+------------+
|  IMDb-id  |         title          | year | IMDb-score |
+-----------+------------------------+------+------------+
| tt1375666 |       Inception        | 2010 |    8.8     |
| tt0816692 |      Interstellar      | 2014 |    8.7     |
| tt4154756 | Avengers: Infinity War | 2018 |    8.4     |
+-----------+------------------------+------+------------+


``` sql
SELECT * FROM movies
WHERE year > 2010
AND IMDb-score >= 8.5;
```

In [11]:
print(select(movies, lambda row: row[2] >= '2010' and row[3] >= '8.5'))

+-----------+--------------+------+------------+
|  IMDb-id  |    title     | year | IMDb-score |
+-----------+--------------+------+------------+
| tt1375666 |  Inception   | 2010 |    8.8     |
| tt0816692 | Interstellar | 2014 |    8.7     |
+-----------+--------------+------+------------+


## Operation: Cartesian-product

In [12]:
def cartesian_product(table_1: PrettyTable, table_2: PrettyTable) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names
    for row1 in table_1.rows:
        for row2 in table_2.rows:
            new_row = row1 + row2
            table_result.add_row(new_row)
    return table_result

``` sql
SELECT * FROM movies, actors;
```

In [13]:
print(cartesian_product(movies, actors))

+-----------+------------------------+------+------------+-----------+-------------------+
|  IMDb-id  |         title          | year | IMDb-score |     id    |        name       |
+-----------+------------------------+------+------------+-----------+-------------------+
| tt0468569 |    The Dark Knight     | 2008 |    9.0     | nm0000288 |   Christian Bale  |
| tt0468569 |    The Dark Knight     | 2008 |    9.0     | nm0000093 |     Brad Pitt     |
| tt0468569 |    The Dark Knight     | 2008 |    9.0     | nm0000138 | Leonardo DiCaprio |
| tt0468569 |    The Dark Knight     | 2008 |    9.0     | nm0000701 |    Kate Winslet   |
| tt0137523 |       Fight Club       | 1999 |    8.8     | nm0000288 |   Christian Bale  |
| tt0137523 |       Fight Club       | 1999 |    8.8     | nm0000093 |     Brad Pitt     |
| tt0137523 |       Fight Club       | 1999 |    8.8     | nm0000138 | Leonardo DiCaprio |
| tt0137523 |       Fight Club       | 1999 |    8.8     | nm0000701 |    Kate Winslet   |

## Operation: project, select and cartesian-product together

``` sql
SELECT title, year 
FROM movies
WHERE year >= 2012;
```

In [14]:
T1 = select(movies, lambda row: row[2] >= '2014')
T2 = project(T1, [1, 2])
print(T2)

+------------------------+------+
|         title          | year |
+------------------------+------+
|      Interstellar      | 2014 |
| Avengers: Infinity War | 2018 |
+------------------------+------+


``` sql
SELECT title, IMDb-score
FROM movies
WHERE IMDb-score >= 8.3
AND IMDb-score < 8.8;
```

In [15]:
T1 = select(movies, lambda row: row[3] >= '8.3' and row[3] < '8.8')
T2 = project(T1, [1, 3])
print(T2)

+------------------------+------------+
|         title          | IMDb-score |
+------------------------+------------+
|         Matrix         |    8.7     |
|      Interstellar      |    8.7     |
| Avengers: Infinity War |    8.4     |
+------------------------+------------+


```sql
SELECT movies.title, persons.name
FROM movies, persons, cast
WHERE movies.id = cast.movie_id
AND persons.id = cast.person_id;
```

In [16]:
T1 = cartesian_product(movies, actors)
T2 = cartesian_product(T1, cast)
T3 = select(T2, lambda row: row[0] == row[6] and row[4] == row[7])
T4 = project(T3, [1, 5])
print(T4)

+-----------------+-------------------+
|      title      |        name       |
+-----------------+-------------------+
| The Dark Knight |   Christian Bale  |
|    Fight Club   |     Brad Pitt     |
|    Inception    | Leonardo DiCaprio |
|     Titanic     | Leonardo DiCaprio |
|     Titanic     |    Kate Winslet   |
+-----------------+-------------------+


## Operation: Joins

### Single-column equi-join

In [17]:
def join(table_1: PrettyTable, table_2: PrettyTable, j1: int, j2: int) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names
    for row1 in table_1.rows:
        for row2 in table_2.rows:
            if row1[j1] == row2[j2]:
                new_row = row1 + row2
                table_result.add_row(new_row)
    return table_result 

``` sql
SELECT * FROM movies, director
WHERE movies.IMDb-id = director.movie_id;
```

In [18]:
print(join(movies, director, 0, 2))

+-----------+-----------------+------+------------+-----------+-------------------+-----------+
|  IMDb-id  |      title      | year | IMDb-score |     id    |        name       |  movie_id |
+-----------+-----------------+------+------------+-----------+-------------------+-----------+
| tt0468569 | The Dark Knight | 2008 |    9.0     | nm0634240 | Christopher Nolan | tt0468569 |
| tt0816692 |   Interstellar  | 2014 |    8.7     | nm0634240 | Christopher Nolan | tt0816692 |
| tt0120338 |     Titanic     | 1997 |    7.9     | nm0000116 |   James Cameron   | tt0120338 |
| tt0499549 |      Avatar     | 2009 |    7.9     | nm0000116 |   James Cameron   | tt0499549 |
+-----------+-----------------+------+------------+-----------+-------------------+-----------+


### Hash-join

In [19]:
def hash_join(table_1: PrettyTable, table_2: PrettyTable, j1: int, j2: int) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names 
    X = dict()
    for id, row1 in enumerate(table_1.rows):
        if row1[j1] not in X:
            X[row1[j1]] = []
        X[row1[j1]].append(id)
    for id, row2 in enumerate(table_2.rows):
        if row2[j2] in X:
            for index in X[row2[j2]]:
                new_row = table_1.rows[index] + table_2.rows[id]
            table_result.add_row(new_row)
    return table_result

In [20]:
print(hash_join(movies, director, 0, 2))

+-----------+-----------------+------+------------+-----------+-------------------+-----------+
|  IMDb-id  |      title      | year | IMDb-score |     id    |        name       |  movie_id |
+-----------+-----------------+------+------------+-----------+-------------------+-----------+
| tt0816692 |   Interstellar  | 2014 |    8.7     | nm0634240 | Christopher Nolan | tt0816692 |
| tt0468569 | The Dark Knight | 2008 |    9.0     | nm0634240 | Christopher Nolan | tt0468569 |
| tt0120338 |     Titanic     | 1997 |    7.9     | nm0000116 |   James Cameron   | tt0120338 |
| tt0499549 |      Avatar     | 2009 |    7.9     | nm0000116 |   James Cameron   | tt0499549 |
+-----------+-----------------+------+------------+-----------+-------------------+-----------+


### Merge-join

In [21]:
def merge_join(table_1: PrettyTable, table_2: PrettyTable, j1: int, j2: int) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names 
    X = sorted([(row[j1], id) for id, row in enumerate(table_1.rows)], key=lambda x: x[0])
    Y = sorted([(row[j2], id) for id, row in enumerate(table_2.rows)], key=lambda x: x[0])
    i, j = len(X), len(Y)
    while i < len(X) and j < len(Y):
        if X[i][0] == Y[j][0]:
            new_row = table_1.rows[X[i][1]] + table_2.rows[Y[j][1]]
            table_result.add_row(new_row)
            i += 1
            j += 1
        elif X[i][0] < Y[j][0]:
            i += 1
        elif X[i][0] > Y[j][0]:
            j += 1
    return table_result

In [22]:
print(merge_join(movies, director, 0, 2))

+---------+-------+------+------------+----+------+----------+
| IMDb-id | title | year | IMDb-score | id | name | movie_id |
+---------+-------+------+------------+----+------+----------+
+---------+-------+------+------------+----+------+----------+
